In [ ]:
#importing the relevant libraries below.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Part I: Hazardous waste datasets - importing, merging, cleaning and the to_ton_converter.

In [ ]:
#Importing the datasets below using pandas' read_excel function: 
haz_waste = pd.read_excel("farlig_avfall.xlsx") #This is the main dataset on waste per facility
facility_1 = pd.read_excel("anlegg_adresse.xlsx") #This is a dataset with address of each facility, exl closed ones
facility_2 = pd.read_excel("anlegg_adresse_v2.xlsx") #This is a dateset later aquired with closed facilities, to be able to complete the following merge.

In [ ]:
#Vertical concatenate facility dateframes into one complete.
facility_complete = pd.concat([facility_2, facility_1]).drop_duplicates().reset_index(drop=True) 

#Fill NaNs for columns "Status" and "Status ansvarlig enhet" with active for active facilties, as this column name was only included in the closed facilites dataset.
facility_complete[["Status", "Status ansvarlig enhet"]]=facility_complete[["Status", "Status ansvarlig enhet"]].fillna("Aktiv") 

In [ ]:
#merging the waste dataset with the facility dataset
waste_merged = pd.merge(haz_waste, facility_complete, how="left", left_on="AnleggNummer", right_on="Anleggsnr.")

In [ ]:
#Checking columns' datatype and non-null count.
waste_merged.info()

In [ ]:
#Checking number of unique values in each column.
waste_merged.nunique()

In [ ]:
#Making a lineplot to see overall development of hazardous waste per year.
sns.lineplot(data=waste_merged, x="År", y="Total mengde", estimator=np.sum) #Estimator=np.sum important to include to get total, otherwise averarge is shown.

In [ ]:
#Scatterplot with waste unit as hue to check reason for spike in 2006-08.
sns.scatterplot(data=waste_merged, x="År", y="Total mengde", hue="Enhet")

In [ ]:
#Defining a converter to convert kilogram and liter to ton.
def to_ton_converter(row, amount_col, unit_col, kg_to_ton=0.001, liter_to_ton=0.0008):
    amount = row[amount_col]
    unit = row[unit_col]
    
    #This checks what kind of unit it is and makes sure the right conversion value is returned.
    if unit == 'Kilogram':
        return amount * kg_to_ton
    elif unit == 'Liter':
        return amount * liter_to_ton
    elif unit == 'Tonn':
        return amount
    else:
        return None
    
#Defining columns to convert, and the columns to store the returned values. This are the tuples the for loop is iterating on next.
columns_to_convert = [
    ("På mellomlager", "Mellomlager i tonn"),
    ("Til gjenvinning i Norge", "Gjenvinning Norge i tonn"),
    ("Til sluttbehandling i Norge", "Sluttbehandling Norge i tonn"),
    ("Til gjenvinning i utlandet", "Gjenvinning utland i tonn"),
    ("Til sluttbehandling i utlandet", "Sluttbehandling utland i tonn"),
    ("Egenbehandlet avfall, forbrenning / pyrolyse med energi-gjenvinning", "Egenbeh forbrenning m energi-gjenvinning i tonn"),
    ("Egenbehandlet avfall, forbrenning / pyrolyse uten energi-gjenvinning", "Egenbeh forbrenning u energigjenvinning i tonn"),
    ("Egenbehandlet avfall, eget godkjent deponi", "Egenbeh eget godkjent deponi i tonn"),
    ("Egenbehandlet avfall, annen behandling", "Egenbeh annen behandling i tonn"),
    ("Total mengde", "Tot mengde i tonn")
]

#Defining the unit column - it is needed to run the to_ton_converter.
unit_col = "Enhet"

#The for loop iterates through each column above.
for amount_col, result_col in columns_to_convert:
    #The apply method passes the lambda function along the rows. Rows are selected by using the axis=1 paramenter.
    waste_merged[result_col] = waste_merged.apply(
        #The lambda function makes sure that the to_ton_converter is run on each row individually.
       lambda row: to_ton_converter(row, amount_col, unit_col), axis=1
    )

In [ ]:
#A lineplot showing total amount of waste per year after converting kilograms and liters to ton.
sns.lineplot(data=waste_merged, x="År", y="Tot mengde i tonn", estimator=np.sum)

In [ ]:
#Making a leaner dataframe and grouping it to fit the prescription drugs data.
waste_groupby_kommune_and_year = waste_merged[["År", "Kommune", "Tot mengde i tonn"]].groupby(["Kommune", "År"]).sum()
waste_groupby_kommune_and_year.reset_index(inplace=True) #Resetting the index to keep the columns.

In [ ]:
#Checking the distribution of waste per "Kommune" per "År" using a histogram.
sns.histplot(data=waste_groupby_kommune_and_year, x="Tot mengde i tonn", binwidth=10000)

Part II: The prescription drugs datasets - importing, cleaning, unstacking and merging

In [ ]:
#Importing Excel files on prescription drugs to dataframes. Type 2 diabetes was a separate dataset.
diabetes_type_II = pd.read_excel("diabetes_type_II.xls", header=7)
prescription_drugs = pd.read_excel("Legemiddelbrukere_ny.xls", header=8)

#Because of merged cells in the original Excel file, using ffill to fill in NaNs below with the last value above.
diabetes_type_II.ffill(inplace=True)
prescription_drugs.ffill(inplace=True)

#Renaming some columns to make more sense of them and for easier merge later.
diabetes_type_II.rename(columns={"År": "År_snitt", "Geografi": "Kommune"}, inplace=True)
prescription_drugs.rename(columns={"År": "År_snitt", "Geografi": "Kommune"}, inplace=True)

#Naming nameless columns - selecting them by the column number.
diabetes_type_II.columns.values[2] = "Type_2-diabetes"
prescription_drugs.columns.values[3] = "Per_1000"

#Unstacking the main prescription drug dataset, so that each prescription drug has its own column. Then it is easier to merge with the Type 2 diabetes dataset. 
prescription_drugs_unstacked = prescription_drugs.set_index(["År_snitt", "Kommune", "Legemiddelgruppe"])["Per_1000"].unstack()
prescription_drugs_unstacked.reset_index(inplace=True) #Reset index to keep individual columns.

#Merging the main prescription drug dataset (allergy, asthma, ADHD, diabetes) with the Type 2 diabetes dataset.
prescription_complete = pd.merge(prescription_drugs_unstacked, diabetes_type_II, how="inner", on=["Kommune", "År_snitt"])

#Prescription data is a three years rolling average and has the year range as column headers. String split used to select last year for easy merge with waste data. 
prescription_complete["År"] = prescription_complete["År_snitt"].str.split("-").str[1]
prescription_complete["År"] = prescription_complete["År"].astype(int) #assuring the values are datatype integer.

#Rename some prescription drug columns for easier edibilty and handling. 
prescription_complete.rename(columns={"Diabetesmedikamenter (A10)": "Diabetes", "Midler mot astma og KOLS (R03 unntatt R03CA)": "Astma_og_KOLS", "Allergimidler (R06A, R01AC, R01AD, R01B, S01G)": "Allergi", "ADHD-midler (C02AC02, N06BA ekskl. N06BA07)": "ADHD"}, inplace=True)

#Selecting the prescription drugs selected for this project into a leaner dataframe. 
prescription_to_correlate = prescription_complete[["År", "Kommune", "Diabetes", "Type_2-diabetes", "Astma_og_KOLS", "Allergi", "ADHD"]]

Part III: Waste and prescription merge, and low-waste/high-waste labels

In [ ]:
#Merging waste df and prescription df to make a new dataset ready to do a correlation analysis:
waste_prescription = pd.merge(waste_groupby_kommune_and_year, prescription_to_correlate, how="inner", on=["Kommune", "År"])

#Converting object type numeric values to float for prescription drug columns - needed to apply mathematical operations on the values.
for col in waste_prescription.columns[3:]: #selecting column four to the end.
    waste_prescription[col] = pd.to_numeric(waste_prescription[col], errors="coerce")

#Removing rows where there is no data on prescription drugs, either because of privacy or other reasons.
waste_prescription = waste_prescription[~waste_prescription.isin(['..']).any(axis=1)]
waste_prescription = waste_prescription[~waste_prescription.isin([':']).any(axis=1)]
waste_prescription = waste_prescription[~waste_prescription.isnull().any(axis=1)]

#Grouping by municipality to calculate the mean, then make a list of municipalities with waste > 1000 per year.
waste_mean_kom = waste_prescription.groupby(by="Kommune")["Tot mengde i tonn"].mean()
waste_mean_kom = pd.DataFrame(waste_mean_kom.reset_index()) 
above_1000_ton = waste_mean_kom.query("`Tot mengde i tonn`> 1000") #querying municipalities with more than 1000 ton waste per year
list_kom=above_1000_ton['Kommune'] #making a list of municipalities from the query aboce.
high_waste_prescription = waste_prescription.query("Kommune in @list_kom") #Using the list to create a high_waste dataframe ...
low_waste_prescription = waste_prescription.query("Kommune not in @list_kom") #... and a low_waste dataframe.

#Creating a label column for low waste municipalities and high waste municipalities.
low_waste_prescription["label"]="low"
high_waste_prescription["label"]="high"

In [ ]:
#Confirming the right datatypes for the dataframe.
print(waste_prescription.dtypes)

In [ ]:
#printing number of unique municipalites in each dataset. 
print(low_waste_prescription["Kommune"].nunique(), high_waste_prescription["Kommune"].nunique())

Part IV: Calculating the correlation coeefficient of each municipality.

In [ ]:
#Importing pearsonr for correlation coefficient and p-value calculation from Scipy Stats.
from scipy.stats import pearsonr

#Defining a prescription_waste_correlator that can be applied to all prescription drug columns, and group the data by municipality. 
def prescription_waste_correlator(data, waste_col, group_col="Kommune", year_col="År"):
    prescription_cols = [col for col in data.columns[3:8]] #Defining the columns to loop through.
    results = pd.DataFrame() #Initializing an empty data frame.

    for prescription_col in prescription_cols: #This is the for loop iterating on each column selected above.
        #Grouping the data by Kommune (group_col) here.
        group_data = data[[group_col, waste_col, prescription_col]].groupby([group_col])
        
        #Initializing lists to add municipalites, correlation coefficients and p_values when looping.
        list_kom = []
        list_corr = []
        list_p_value = []

        #This for loop iterates on each group (municipality) of each column.
        for name, group in group_data: 
            #Calculating the correlation coeefficient and the p-value for each municipality. 
            corr_val, p_value = pearsonr(group[waste_col], group[prescription_col])
            #Appending the results in each list. 
            list_kom.append(name)
            list_corr.append(corr_val)
            list_p_value.append(p_value)
        
        #Creating a dataframe to combine the lists.
        prescription_results = pd.DataFrame({
            group_col: list_kom,
            "Legemiddel": prescription_col,
            "Korrelasjon": list_corr,
            "P-value": list_p_value
        })
        
        #Concatenating for each run of the loop, and ignoring the pre concatenating indexes.
        results = pd.concat([results, prescription_results], ignore_index=True)

    return results

In [ ]:
#Running the prescription_waste_correlator function on the high_waste_prescription dataframe. 
correlation_results = prescription_waste_correlator(high_waste_prescription, "Tot mengde i tonn")

#Only select results with a p-value of < 0,05.
p_value_0_05 = correlation_results.query("`P-value` < 0.05")

In [ ]:
#Checking the distribution of the correlation results using histogram.
sns.histplot(data=correlation_results, x="Korrelasjon")

In [ ]:
#Boxplot to show the distribution of different prescription drugs.
ax = sns.boxplot(data=p_value_0_05, y="Korrelasjon", x="Legemiddel")
ax.set_xticklabels(ax.get_xticklabels(), rotation=20) #Rotating the x axis a bit to increase legibility. 
plt.show()


Part V: High waste vs low waste municipalities

In [ ]:
#Concatenating high_waste with low_waste to create a full dataset that now includes labels for high and low waste.
combined_df = pd.concat([high_waste_prescription, low_waste_prescription])

#Loopig through the prescription drug columns creating a lineplot for each, using the high/low label with the hue paramenter.
for legemiddel in combined_df.columns[3:8]:
    
    plt.figure()
    
    sns.lineplot(data=combined_df, x="År", y=legemiddel, hue="label", estimator=np.mean)
    plt.title(legemiddel)
plt.show()

In [ ]:
#Creating a histograph to see the allergy distribution for high vs low waste municipalities.
sns.histplot(data=combined_df, x="Allergi", hue="label")

Part VI: Regression on a logarithmic scale to further investigate allergy findings

In [ ]:
#Using a scatterplot to look for a pattern between allergy prescription rates (y) and hazardous waste (x)
sns.scatterplot(data=waste_prescription, y="Allergi", x="Tot mengde i tonn", hue="År", palette="plasma")

In [ ]:
sns.scatterplot(data=waste_prescription, y="Allergi", x="Tot mengde i tonn", hue="År", palette="plasma")
plt.xscale("log") #Adding a logarithmic scale to the x axis (waste)

In [ ]:
#also importing spearmanr for rank correlation as my dataset is skewed.
from scipy.stats import spearmanr

prescription = "Allergi" #defining prescription as a variable to easily switch between different prescription drugs.
correlator = pearsonr #defining correlator as a variable to easily switch between pearsonr and spearmanr.

#Creating logarithmic columns for waste and prescription.
waste_prescription["log_waste"] = np.log10(waste_prescription["Tot mengde i tonn"]+1)
waste_prescription["log"+prescription] = np.log10(waste_prescription[prescription]+1)

#Referencing the lmplot I want to run in the variable g.
g = sns.lmplot(data=waste_prescription, y="log"+prescription, x="log_waste")

#Defining a function that runs pearsonr, or spearmanr, and annotate the values to a plot. This is stolen from Stack Overflow.
def annotate(data, **kws):
    r, p = correlator(data["log"+prescription], data['log_waste']) #here I am running pearsonr, or spearmanr, on the data
    ax = plt.gca()
    ax.text(.8, .05, "r={:.2f}, p={:.2g}".format(r, p), #defining the placement of the annotation, down to the right.
            transform=ax.transAxes)
g.map_dataframe(annotate) #adding the annotation function to the plot.

plt.show()

In [ ]:
#Adding hue="År" to the lineplot to visualize regression per year.
sns.lmplot(data=waste_prescription, y="log"+prescription, x="log_waste", hue="År", palette="plasma")

Part VII: Correlation per year

In [ ]:
#Defining a variable for unique years in the year column.
years = waste_prescription["År"].unique()

#Creating a dictionary to store the results for each iteration of the loop.
results = {
    "year": [], 
    "corr": [], 
    "p_value": []
    }

#A for loop that iterates on each year.
for year in years:
    #The data is first filtered for the year the loop is iterating on. 
    waste_prescription_years = waste_prescription[waste_prescription["År"] == year]
    
    #Then it is calculating the correlation (Pearson's or Spearman's depending on input above) and p-value for that year.
    corr, p_value = correlator(waste_prescription_years["Allergi"], waste_prescription_years["log_waste"])
    
    #The results are appended in the dictionary defined above.
    results["year"].append(year)
    results["corr"].append(corr)
    results["p_value"].append(p_value)

#The dictionary is converted into a data frame.
results_df = pd.DataFrame(results)

#Sorting the results by years.
results_df.sort_values(by="year", inplace=True)

print(results_df)


In [ ]:
sns.lineplot(data=results_df, x="year", y="corr") #a lineplot of correlation over time.

Part VIII: Creating a 3D scatter plot using plotly and with the help of ChatGPT.

In [ ]:

#Importing the libraries needed to create the 3D plot.

import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

plot_data = waste_prescription

#Define the differenct axes of the 3D plot
x_column = "log_waste"
y_column = "År"
z_column = "logAllergi"

#Preparing the data for linear regression fitting
X = plot_data[[x_column, y_column]]
y = plot_data[z_column]

#Fitting the linear regression model on the data
model = LinearRegression()
model.fit(X, y)

# Creating an array for predictions for the regression line
X_pred = pd.DataFrame({
    x_column: np.linspace(X[x_column].min(), X[x_column].max(), 100),
    y_column: np.linspace(X[y_column].min(), X[y_column].max(), 100)
})
y_pred = model.predict(X_pred)

# Creating the 3D figure
fig = go.Figure()

# Add scatter plot - waste data
fig.add_trace(go.Scatter3d(x=plot_data[x_column], y=plot_data[y_column], z=plot_data[z_column],
                           mode='markers', marker=dict(size=2, color='blue',)),)

# Add line plot - prediction data, as defined above
fig.add_trace(go.Scatter3d(x=X_pred[x_column], y=X_pred[y_column], z=y_pred,
                           mode='lines', line=dict(color='red', width=2)))

# Updatingn the plot appearance
fig.update_layout(title='3D Regression Line Plot',
                  scene=dict(
                      xaxis_title=x_column,
                      yaxis_title=y_column,
                      zaxis_title=z_column))

# And this is showing the figure.
fig.show()


Bonus: Let's see if I can do a Principal Component Analysis (PCA) on the data - ChatGPT has been helpful also here

In [ ]:
#importing libraries needed rom sklearn, StandardScaler to standardize the data,  PCA to run the PCA.
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#grouping by municipality to get on PCA value per municipality.
new_pca = waste_prescription.groupby("Kommune").mean().reset_index()

#selecting the numerical values that goes into the PCA, and standardize it with the StandardScaler.
x = new_pca.iloc[:,3:-1].values
x = StandardScaler().fit_transform(x)

pca = PCA(n_components=3) #defining number of components for PCA.
principal_components = pca.fit_transform(x) #running the pca on the standardized data.
principal_df = pd.DataFrame(data=principal_components, columns=["PC_1", "PC_2", "PC_3"]) #Creating a data frame of the results.

#concatenating pca result data frame with original df on rows so that PCA values are alligned with the right municipality.
final_df = pd.concat([new_pca, principal_df], axis=1)

In [ ]:
#creating a 3D scatter plot to display PC_1, PC_2 and PC_3.
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

#Defining the data for the scatter plot.
sc = ax.scatter(final_df["PC_1"], final_df["PC_2"], final_df["PC_3"], c='r', marker='o')

#Annotate each point with the municipality name (if applicable)
for i, municipality in enumerate(final_df["Kommune"]):
    ax.text(final_df.loc[i, "PC_1"], final_df.loc[i, "PC_2"], final_df.loc[i, "PC_3"], municipality)

#Set labels and plot title.
ax.set_xlabel("PC_1")
ax.set_ylabel("PC_2")
ax.set_zlabel("PC_3")
ax.set_title('PCA: Hazardous waste and prescription drugs')

plt.show()

In [ ]:
#Now I want to see the weights the different input in the PCA has on the different principal components. 
df_selected = new_pca.iloc[:,3:-1] #defining a variable for the columns used in the PCA.

#Extract loadings/weights as loadings
loadings = pca.components_

#Create a data frame for loadings, setting each PC as columns and the original columns, that went into the PCA, as rows.
loadings_df = pd.DataFrame(loadings.T, columns=['PC_1', 'PC_2', 'PC_3'], index=df_selected.columns)
print(loadings_df)

# Plot loadings for each principal component
fig, axes = plt.subplots(1, 3, figsize=(14, 6))

for i, ax in enumerate(axes): #for loop to create a bar plot for each PC.
    loadings_df.iloc[:, i].plot(kind='bar', ax=ax)
    ax.set_title(f'Loadings for Principal Component {i+1}')
    ax.set_ylabel('Loading Score')
    ax.set_xlabel('Features')

plt.tight_layout()
plt.show() #Display all three plots.